**Cargad las librerías de numpy, pandas y matplotlib con el nombre de siempre**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

**Leed la data  `cleveland_heart_4.csv` cargándolo como un data frame de pandas. Ajustad el argumento na_values='?' y header=None**

In [ ]:
clev_data=pd.read_csv('../../datasets/cleveland_heart_4.csv', na_values='?', header=None)

** Reescribid el nombre de las columnas a:'age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','diagnosis'**

In [ ]:
header_names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','diagnosis']
clev_data.columns = header_names

** Cread una matriz numpy de features llamada `X` y un array numpy de targets llamado `y` **

In [ ]:
X = clev_data.drop(columns=['diagnosis']).values
y = clev_data['diagnosis'].values

** Como podéis ver, el vector `y` contiene los valores del target, pero éstos están en tipo texto. Scikit solo trabaja con datos numéricos. Por tanto, redefinid `y`, convirtiendo estos targets no numéricos a numéricos usando la clase `LabelEncoder`, que como vimos, se encuentra en el módulo `preprocessing` de scikit**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

** Nuestra dataset contiene variables categóricas, tienen NaNs y además posee muchas features. Por tanto, cread un pipeline que implemento los siguientes pasos: **

1. **Imputación de los NaNs de tal forma que sustituya los NaNs en cada columna por la mediana**
2. **Ortogonalización de las variables categóricas de tal forma que cada categoría sea una columna nueva. Las variables categóricas son: "sex", "cp", "fbs", "restecg", "exang", "slope", "ca" y "thal"**
3. **Reescalar el rango de los datos a un rango [0,1]**
4. **Una feature selection que seleccione los mejores features a partir de un árbol de decisión **

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, Imputer, StandardScaler
from sklearn.feature_selection import SelectFromModel, SelectKBest,f_classif
from sklearn.tree import DecisionTreeClassifier

oHe = OneHotEncoder(categorical_features=[1,2,5,6,8,10,11,12], sparse=False)
ss = MinMaxScaler()
#ss = StandardScaler()
imp= Imputer(strategy='median')
tree_clf = DecisionTreeClassifier(random_state=5)
feat_sel = SelectFromModel(tree_clf)
#feat_sel = SelectKBest(f_classif, k=10)
pip = Pipeline([('imp', imp),('oHe',oHe),('ss',ss),('feat_sel',feat_sel)])

**Comprobad que ha funcionado haciendo un `fit_transform` sobre X e y. Mostrad el output y el tamaño de los datos transformados**

In [ ]:
print(pip.fit_transform(X,y))
print(pip.fit_transform(X,y).shape)

** Volved a definir el mismo pipeline de antes, pero incorporando ahora un clasificador a vuestro gusto ( Yo voy a coger SVM) **

In [ ]:
from sklearn.svm import SVC

clf = SVC(class_weight='balanced')
pip = Pipeline([('imp', imp),('oHe',oHe),('ss',ss),('feat_sel',feat_sel), ('clf', clf) ])

** Cread una partición de los datos y evaluad la performance del modelo usando el pipeline anterior **

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

pip.fit(X_train,y_train)
print(pip.score(X_test, y_test))

** Ahora evaluad el modelo mediante la técnia de cross-validation. Podéis elegir el esquema que queráis, por ejemplo, un 10-fold cross-validation **

In [ ]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

rcv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)

In [ ]:
pip.named_steps

In [ ]:
 np.mean(cross_val_score(pip, X, y,  cv=rcv))

** Seguramente, el rendimiento obtenido se pueda mejorar cambiando los hiperparámetros del clasificador y el threshold del feature selection. Mirad esto, sobre la partición X_train, X_test, y_train, y_test de antes, usando la función `GridSearchCV` o `RandomSearchCV` para diferentes valores de los hiperparámetro del clasificador y threshold de la feature selection. Cuidado con el grid que definais y el número de folds que uséis, ya que puede hacer el problema muy time-consuming **

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

skf_1 = StratifiedKFold(n_splits=5, random_state=0)

param_grid = {'feat_sel__threshold': [0.001,0.01,0.1, None], 'clf__C':[0.01,0.1,1,10]} 
grid =GridSearchCV(pip, param_grid=param_grid, cv=skf_1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_score_

In [ ]:
print(grid.score(X_test, y_test))

**Ahora, meted esto dentro dentro un cross-validation y calculad la performance**

In [ ]:
skf_2 = StratifiedKFold(n_splits=5, random_state=10)

print(np.mean(cross_val_score(grid, X, y, cv=skf_2)))

**Extra: Repetid lo mismo de antes con diferentes clasificadores, para que podamos hacernos una idea de cuál es el mejor**

Sin gridSearch

In [ ]:
# Cargo los clasificadores
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Hago una lista con los objetos de estos clasificadores y los nombres, sólo para hacer un plot
list_clas_names = ['Logistic Regression', 'RandomForest', 'SVM', 'KNN']
list_clf = [LogisticRegression(), RandomForestClassifier(), SVC(), KNeighborsClassifier()]

#Defino el mismo esquema de cross-validation
skf_2 = StratifiedKFold(n_splits=5, random_state=10)

for i,clf in enumerate(list_clf):
    
    pip = Pipeline([('imp', imp),('oHe',oHe),('ss',ss),('feat_sel',feat_sel), ('clf', clf)])
    
    res = cross_val_score(pip, X, y, cv=skf_2)
    #print(np.mean(cross_val_score(pip, X, y, cv=skf_2)))
    plt.bar(i, np.mean(res), yerr=np.std(res))

plt.legend(list_clas_names)
plt.ylim([0.7,1.0])
pass

Con busqueda sobre los hiperparámetros. Hacerlo sobre un grid podría llevar mucho tiempo. Por ello, voy a hacer una búsqueda aleatoria usano `RandomizedSearchCV` con 10 iteraciones por cada clasificador

In [ ]:
# Cargo el módulo de RandomSeachCV
from sklearn.model_selection import RandomizedSearchCV

# Hago una lista de dictionarios para saber los hiperparámetros de cada clasifiador sobre el que buscar
list_params = [{'clf__C':[0.01, 0.01,0.1,1,10], 'clf__penalty':['l1', 'l2']},
              { 'clf__max_depth':np.arange(1,10), 'clf__criterion':['gini', 'entropy']},
              {'clf__C':[0.01,0.1,1,10], 'clf__kernel': [ 'linear', 'poly', 'rbf', 'sigmoid']},
              {'clf__n_neighbors':[1,5,10,15, 20], 'clf__weights':['uniform','distance']} ]

#Defino el mismo esquema de cross-validation
skf_inner = StratifiedKFold(n_splits=5, random_state=0)
skf_outer = StratifiedKFold(n_splits=5, random_state=10)

for i,clf in enumerate(list_clf):
    
    pip = Pipeline([('imp', imp),('oHe',oHe),('ss',ss),('feat_sel',feat_sel), ('clf', clf)])
    
    grid = RandomizedSearchCV(pip, list_params[i], cv=skf_inner, random_state=100)
    res = cross_val_score(grid, X, y, cv=skf_outer)
    #print(np.mean(cross_val_score(pip, X, y, cv=skf_2)))
    plt.bar(i, np.mean(res), yerr=np.std(res))

plt.legend(list_clas_names)
plt.ylim([0.7,1.0])
pass